# Brain Tumor MRI Torch Conv2d

In [151]:
import os
import random
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time

from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader

In [152]:
data_dir = './archive/Training'
test_dir = './archive/Testing'
classes = os.listdir(data_dir)
print(classes)
print(len(classes))

['glioma', 'meningioma', 'notumor', 'pituitary']
4


In [153]:
train_transform=transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
])

In [154]:
dataset = ImageFolder(data_dir, transform=train_transform)
testset = ImageFolder(test_dir, transform=train_transform)

In [155]:
img, label = dataset[100]
print(img.shape)

torch.Size([3, 32, 32])


In [156]:
def show_image(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1,2,0))

In [157]:
torch.manual_seed(10)
test_size = len(testset)
train_size = len(dataset)

In [158]:
train_ds = dataset
test_ds = testset
len(train_ds), len(test_ds)   

(5712, 1311)

In [159]:
batch_size = 64

train_dataloader = DataLoader(train_ds, batch_size=64)
test_dataloader = DataLoader(test_ds, batch_size=64)

In [160]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout()
        )
        self.fc3 = nn.Linear(256, 4)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [161]:
model = VGG16()
#model.cuda()

In [162]:
model

VGG16(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(128, 

for images, labels in train_loader:
    out = model(images)
    print('images.shape:', images.shape)
    print('out.shape:', out.shape)
    print('out[0]:', out[0])
    break

In [163]:
epoch = 10
train_step = 0
test_step = 0
device=torch.device("cuda:0")
 
vgg16 = VGG16().to(device)  # 实例化网络
loss_fn = nn.CrossEntropyLoss().to(device)  
 
learning_rate = 1e-2
# optimizer = optim.SGD(vgg16.parameters(), lr=learning_rate, momentum=0.9)
optimizer = optim.Adam(vgg16.parameters(), lr=learning_rate)
 
losses = []  # 用于存储损失值，便于后面画损失变化曲线图

In [164]:
t = time.strftime('%Y-%m-%d_%H-%M-%S')  # 切记中间符号不能用冒号，不然会报错
train_data_size = len(train_ds)
test_data_size = len(test_ds)
print('The size of train_data:{}'.format(train_data_size))
print('The size of train_data:{}'.format(test_data_size))

The size of train_data:5712
The size of train_data:1311


for i in range(epoch):
    print('-----epoch{}-----'.format(i))
 
    # 训练步骤开始
    vgg16.train()
    for data in train_dataloader:
        images, targets = data
        images = images.to(device)
        targets = targets.to(device)
        output = vgg16(images)
        loss = loss_fn(output, targets)
 
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_step += 1
        if train_step % 50 == 0:
            print('train_step:{},loss:{}'.format(train_step, loss.item()))
            losses.append(loss.item())
 
    # 测试步骤开始
    vgg16.eval()
    accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            images, targets = data
            images = images.to(device)
            targets = targets.to(device)
            output = vgg16(images)
            current_acc = (output.argmax(1) == targets).sum()
            accuracy += current_acc
 
    acc = accuracy / test_data_size
    print('-------eval accuracy:{}'.format(acc))
 
    torch.save(vgg16, 'checkpoints/vgg16_{}.pth'.format(i))
    print('The model has been saved')

In [165]:
#超参数设置
batch_size  = 64 # 每次训练的数据量
learn_rate = 0.01 # 学习率
step_size = 10 # 控制学习率变化
epuch_num = 10 # 总的训练次数
num_print = 10 # 每n个batch打印一次

# 生成驱动器
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 利用驱动器驱动模型
model = VGG16().to(device=device)

# 损失函数
get_loss = nn.CrossEntropyLoss()

# SGD优化器  第一个参数是输入需要优化的参数,第二个是学习率，第三个是动量，大致就是借助上一次导数结果，加快收敛速度。
optimizer = optim.SGD(model.parameters(),lr=learn_rate,momentum=0.8,weight_decay=0.001)

# 动态调整学习率 StepLR 是等间隔调整学习率，每step_size 令lr=lr*gamma
# 学习率衰减，随着训练的加深，目前的权重也越来越接近最优权重，原本的学习率会使得，loss上下震荡，逐步减小学习率能加快收敛速度。
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=step_size,gamma=0.5,last_epoch=-1)

loss_list=[]
start=time.time()

for epoch in range(epuch_num):
    running_loss = 0.0
    # enumerate()是python自带的函数，用于迭代字典。参数1，是需要迭代的对象，第二参数是迭代的起始位置
    for i,(inputs,labels) in enumerate(train_dataloader,0):
        inputs,labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()# 将梯度初始化为0
        outputs = model(inputs)# 前向传播求出预测的值
        loss = get_loss(outputs,labels).to(device)# 求loss,对应loss +=

        loss.backward() # 反向传播求梯度
        optimizer.step() # 更新所有参数

        running_loss += loss.item()# loss是张量，访问值时需要使用item()
        loss_list.append(loss.item())

        if i % num_print == num_print - 1:
            print('[%d epoch, %d] loss: %.6f' % (epoch + 1, i + 1, running_loss / num_print))
            running_loss = 0.0
    lr = optimizer.param_groups[0]['lr']# 查看目前的学习率
    print('learn_rate : %.15f'% lr)
    scheduler.step()# 根据迭代epoch更新学习率
    
    model.eval()
    accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            images, targets = data
            images = images.to(device)
            targets = targets.to(device)
            output = vgg16(images)
            current_acc = (output.argmax(1) == targets).sum()
            accuracy += current_acc
 
    acc = accuracy / test_data_size
    print('-------eval accuracy:{}'.format(acc))
 
    torch.save(vgg16, 'checkpoints/vgg16_{}.pth'.format(i))
    print('The model has been saved')


end = time.time()
print('time:{}'.format(end-start))

torch.save(model,'./model.pth')

[1 epoch, 10] loss: 0.376433
[1 epoch, 20] loss: 0.000126
[1 epoch, 30] loss: 1.966679
[1 epoch, 40] loss: 0.001867
[1 epoch, 50] loss: 2.246227
[1 epoch, 60] loss: 0.101388
[1 epoch, 70] loss: 1.871926
[1 epoch, 80] loss: 1.131658
[1 epoch, 90] loss: 0.140920
learn_rate : 0.010000000000000
-------eval accuracy:0.23340961337089539
The model has been saved
[2 epoch, 10] loss: 2.145489
[2 epoch, 20] loss: 0.000001
[2 epoch, 30] loss: 74.806243
[2 epoch, 40] loss: 0.000000
[2 epoch, 50] loss: nan
[2 epoch, 60] loss: nan
[2 epoch, 70] loss: nan
[2 epoch, 80] loss: nan
[2 epoch, 90] loss: nan
learn_rate : 0.010000000000000
-------eval accuracy:0.242562934756279
The model has been saved
[3 epoch, 10] loss: nan
[3 epoch, 20] loss: nan
[3 epoch, 30] loss: nan
[3 epoch, 40] loss: nan
[3 epoch, 50] loss: nan
[3 epoch, 60] loss: nan
[3 epoch, 70] loss: nan
[3 epoch, 80] loss: nan
[3 epoch, 90] loss: nan
learn_rate : 0.010000000000000
-------eval accuracy:0.22273074090480804
The model has been sav

KeyboardInterrupt: 